In [1]:
import os

In [2]:
%pwd

'c:\\Users\\User\\Desktop\\MLOps Project\\End-to-End-ML-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\User\\Desktop\\MLOps Project\\End-to-End-ML-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
 root_dir : Path
 source_URL : str
 local_data_file: Path
 unzip_dir: Path

In [6]:
%ls

 Volume in drive C has no label.
 Volume Serial Number is 9601-F3BC

 Directory of c:\Users\User\Desktop\MLOps Project\End-to-End-ML-Project

01/28/2024  07:18 PM    <DIR>          .
01/28/2024  07:18 PM    <DIR>          ..
01/28/2024  11:59 AM             3,238 .gitignore
01/28/2024  12:55 PM                 0 app.py
01/28/2024  12:55 PM    <DIR>          config
01/28/2024  07:18 PM    <DIR>          Data
01/28/2024  11:59 AM             1,089 LICENSE
01/28/2024  04:33 PM    <DIR>          logs
01/28/2024  07:32 PM                47 main.py
01/28/2024  06:45 PM                 9 params.yaml
01/28/2024  06:13 PM               517 README.md
01/28/2024  01:19 PM               138 requirements.txt
01/28/2024  06:31 PM    <DIR>          research
01/28/2024  06:44 PM               379 schema.yaml
01/28/2024  07:20 PM               771 setup.py
01/28/2024  01:25 PM    <DIR>          src
01/28/2024  12:55 PM             1,266 template.py
01/28/2024  12:55 PM    <DIR>          templates
     

In [7]:
from src.MLProject.constants import *
from src.MLProject.utils.common import read_yaml,create_directories

In [9]:
class ConfigurationManager:
 def __init__(
   self,
   config_filepath = CONFIG_FILE_PATH,
   params_filepath = PARAMS_FILE_PATH,
   schema_filepath = SCHEMA_FILE_PATH):
  
    self.config = read_yaml(config_filepath)
    self.params = read_yaml(params_filepath)
    self.schema = read_yaml(schema_filepath)

    create_directories([self.config.artifacts_root])


 def get_data_ingestion_config(self) -> DataIngestionConfig:
   config = self.config.data_ingestion

   create_directories([config.root_dir])

   data_ingestion_config = DataIngestionConfig(
       root_dir=config.root_dir,
       source_URL=config.source_URL,
       local_data_file=config.local_data_file,
       unzip_dir=config.unzip_dir 
   )

   return data_ingestion_config


In [11]:
import os
import urllib.request as request
import zipfile
from MLProject import logger
from MLProject.utils.common import get_size

In [12]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [21]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-01-28 20:04:16,697: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-28 20:04:16,699: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-28 20:04:16,702: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-01-28 20:04:16,704: INFO: common: created directory at: artifacts]
[2024-01-28 20:04:16,706: INFO: common: created directory at: artifacts/data_ingestion]
[2024-01-28 20:04:18,642: INFO: 1543902713: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 100356
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c7eb16441bd00dc318bbc922b362147e8ed3565dc5d64a9c26ca21170ebbd49e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 5E88:CEAA3:9F6CA:C2246:65B665EA
Accept-Ranges: bytes
Date: Sun, 28 J